In [5]:
pip install google

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install numpy
!pip install scikit-surprise
import os
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse
from collections import defaultdict
from surprise import KNNBasic
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise import AlgoBase

     |████████████████████████████████| 11.8 MB 6.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630165 sha256=1121a4c49ae0a433a0f3677f8bad1f89f90b581fd056fff549a8f189bf79751f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [10]:
ratings = pd.read_csv('/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
ratings_map = {}
ratings_map['itemID'] = list(ratings.movieId)
ratings_map['userID'] = list(ratings.userId)
ratings_map['rating'] = list(ratings.rating)

df = pd.DataFrame(ratings_map)
df.shape

(100836, 3)

In [12]:
from surprise.model_selection import KFold
rr = Reader(rating_scale=(0.5, 5.0))
rating_map_keys = ['userID', 'itemID', 'rating']
filter = df[rating_map_keys]
data = Dataset.load_from_df(filter, rr)

kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7f6c557623d0>

In [13]:
trainset, testset = train_test_split(data, test_size=0.2)
algo = SVD(n_factors=30, n_epochs=20, lr_all=0.008, reg_all=0.08)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.8641


0.8641432622800898

In [15]:
algo1 = KNNBasic(n_factors=30, n_epochs=20)
predictions2 = algo1.fit(trainset).test(testset)
accuracy.rmse(predictions2)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9421


0.9420790684480527

In [18]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):

    userid_to_estimate_map = defaultdict(list)
    for user_id, _, ratings_true, estimate_value, _ in predictions:
        userid_to_estimate_map[user_id].append((estimate_value, ratings_true))

    precisions = dict()
    recalls = dict()
    for user_id, user_ratings in userid_to_estimate_map.items():

        user_ratings.sort(key=lambda x: x[0], reverse=True)

        n_rel = sum((ratings_true >= threshold) for (_, ratings_true) in user_ratings)

        n_rec_k = sum((estimate_value >= threshold) for (estimate_value, _) in user_ratings[:k])

        n_rel_and_rec_k = sum(((ratings_true >= threshold) and (estimate_value >= threshold))
                              for (estimate_value, ratings_true) in user_ratings[:k])

        precisions[user_id] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        recalls[user_id] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [19]:
kf = KFold(n_splits=4)

i = 1
for trainset, testset in kf.split(data):
    print("Split:", i)
    predictions = algo1.fit(trainset).test(testset)
    accuracy.rmse(predictions, verbose=True)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    print("Precision:", sum(prec for prec in precisions.values()) / len(precisions))
    print("Recall:", sum(rec for rec in recalls.values()) / len(recalls))

Split: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9490
Precision: 0.7934699453551924
Recall: 0.25461574366041145
Split: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9518
Precision: 0.7769398907103835
Recall: 0.26155487184094406
Split: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9511
Precision: 0.7766393442622963
Recall: 0.2665051902172836
Split: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9531
Precision: 0.7621038251366131
Recall: 0.26986408655957206


In [20]:
def getpreds(predictions):
    
    fin = defaultdict(list)    
    for user_id, id, ratings_true, estimate_value, _ in predictions:
        fin[user_id].append((id, estimate_value))

    for user_id, user_ratings in fin.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    return fin

In [22]:
trainset = data.build_full_trainset()
algo1.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo1.test(testset)
all_pred = getpreds(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [23]:
n = 10

for user_id, user_ratings in all_pred.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    all_pred[user_id] = user_ratings[:n]

In [24]:
tmp = pd.DataFrame.from_dict(all_pred)
tmp_transpose = tmp.transpose()

In [25]:
res = []
for user_id,user_ratings in all_pred.items():
  res.append(tmp_transpose.loc[user_id])

In [26]:
recomml = []
for i in res:
  recommended_movie_ids=[]
  for x in range(0, n):
    recommended_movie_ids.append(i[x][0])
  recomml.append(recommended_movie_ids)

In [28]:
movies = pd.read_csv('/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA/movies.csv')
finall = []
for i in recomml:
  df = movies[movies['movieId'].isin(i)]
  temp = df['title'].tolist()
  finall.append(temp)

In [29]:
fin = pd.DataFrame(finall)

In [30]:
fin.to_csv('/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA/file1.csv',index = False)

In [32]:
r = pd.read_csv('/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA/file1.csv')
r

,0,1,2,3,4,5,6,7,8,9
0,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Galaxy of Terror (Quest) (1981),Alien Contamination (1980),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
1,Far From Home: The Adventures of Yellow Dog (1...,Lassie (1994),It's My Party (1996),Children of the Corn IV: The Gathering (1996),Lesson Faust (1994),I'm the One That I Want (2000),Elling (2001),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),Black Mirror
2,It's My Party (1996),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Elling (2001),My Sassy Girl (Yeopgijeogin geunyeo) (2001),Strictly Sexual (2008),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),Black Mirror
3,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),Galaxy of Terror (Quest) (1981),Alien Contamination (1980),My Sassy Girl (Yeopgijeogin geunyeo) (2001),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
4,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Galaxy of Terror (Quest) (1981),Elling (2001),Alien Contamination (1980),Dylan Moran: Monster (2004),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
...,...,...,...,...,...,...,...,...,...,...
605,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Galaxy of Terror (Quest) (1981),Alien Contamination (1980),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
606,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Galaxy of Terror (Quest) (1981),Alien Contamination (1980),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
607,Lesson Faust (1994),"Assignment, The (1997)",Mephisto (1981),I'm the One That I Want (2000),Galaxy of Terror (Quest) (1981),Alien Contamination (1980),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
608,Lesson Faust (1994),"Assignment, The (1997)",Bent (1997),Mephisto (1981),I'm the One That I Want (2000),Elling (2001),Dylan Moran: Monster (2004),Bill Hicks: Revelations (1993),The Jinx: The Life and Deaths of Robert Durst ...,Black Mirror
